In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
train=pd.read_csv('Train_aBjfeNk.csv')
test=pd.read_csv('Test_LqhgPWU.csv')
sub=pd.read_csv('sample_submission_wyi0h0z.csv')

In [4]:
train.shape, test.shape

((8068, 11), (2627, 10))

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [6]:
test.shape[0]

2627

In [7]:
(train['ID'].isin(test['ID'])).sum()

2332

In [8]:
(test.ID.isin(train.ID)==False).sum()

295

In [9]:
print('% of test IDs which are already available in train:', 2332/2627)
print('% of test IDs to be predicted:', 295/2627)

% of test IDs which are already available in train: 0.8877046060144652
% of test IDs to be predicted: 0.11229539398553483


In [10]:
print(train['Work_Experience'].median())
print(train['Family_Size'].median())

1.0
3.0


In [11]:
# Fill NaN
train['Work_Experience']=train['Work_Experience'].fillna(1)
train['Family_Size']=train['Family_Size'].fillna(3)

test['Work_Experience']=test['Work_Experience'].fillna(1)
test['Family_Size']=test['Family_Size'].fillna(3)

In [12]:
#Label encode the cat_cols in train

In [13]:
list(train.select_dtypes(include='object').columns)

['Gender',
 'Ever_Married',
 'Graduated',
 'Profession',
 'Spending_Score',
 'Var_1',
 'Segmentation']

In [14]:
categorical_columns = ['Gender','Ever_Married','Graduated','Profession','Spending_Score','Var_1']

In [15]:
for c in categorical_columns:
    mapper={y:x for x,y in enumerate(train[c].unique())}
    train[c]=train[c].map(mapper)
    
    mapper={y:x for x,y in enumerate(test[c].unique())}
    test[c]=test[c].map(mapper)

In [16]:
train['Segmentation']=train['Segmentation'].map({'A':0, 'B':1, 'C':2, 'D':3})

In [17]:
#Make two test dataframes, one to be filled from train data and other to be predicted

In [18]:
true_test = test[test.ID.isin(train.ID)==False]
true_test

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
6,459005,1,0,61,0,5,5.0,0,3.0,0
19,459045,0,0,88,0,7,1.0,1,4.0,0
32,459090,1,1,31,1,6,1.0,0,2.0,0
38,459116,1,0,60,0,6,2.0,1,5.0,0
43,459121,0,0,51,0,6,3.0,1,6.0,0
...,...,...,...,...,...,...,...,...,...,...
2607,467913,1,0,48,0,6,3.0,0,1.0,0
2612,467930,1,0,45,0,8,4.0,1,2.0,0
2616,467938,1,0,46,0,8,5.0,0,1.0,5
2624,467960,0,1,53,0,8,1.0,0,2.0,0


In [19]:
false_test=pd.merge(test[['ID']], train[['ID','Segmentation']], on='ID', how ='left')
false_test

,ID,Segmentation
0,458989,1.0
1,458994,2.0
2,458996,0.0
3,459000,2.0
4,459001,2.0
...,...,...
2622,467954,3.0
2623,467958,0.0
2624,467960,NaN
2625,467961,1.0


In [20]:
false_test[false_test.Segmentation.isnull()]

,ID,Segmentation
6,459005,NaN
19,459045,NaN
32,459090,NaN
38,459116,NaN
43,459121,NaN
...,...,...
2607,467913,NaN
2612,467930,NaN
2616,467938,NaN
2624,467960,NaN


In [21]:
false_test=pd.DataFrame.dropna(false_test)
false_test

,ID,Segmentation
0,458989,1.0
1,458994,2.0
2,458996,0.0
3,459000,2.0
4,459001,2.0
...,...,...
2620,467949,3.0
2621,467950,3.0
2622,467954,3.0
2623,467958,0.0


In [22]:
train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,0,0,22,0,0,1.0,0,4.0,0,3
1,462643,1,1,38,1,1,1.0,1,3.0,0,0
2,466315,1,1,67,1,1,1.0,0,1.0,1,1
3,461735,0,1,67,1,2,0.0,2,2.0,1,1
4,462669,1,1,40,1,3,1.0,2,6.0,1,0


In [23]:
train.head(2)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,0,0,22,0,0,1.0,0,4.0,0,3
1,462643,1,1,38,1,1,1.0,1,3.0,0,0


In [24]:
true_test.head(2)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
6,459005,1,0,61,0,5,5.0,0,3.0,0
19,459045,0,0,88,0,7,1.0,1,4.0,0


In [25]:
false_test.head(2)

,ID,Segmentation
0,458989,1.0
1,458994,2.0


In [26]:
X_train=train.drop(['ID', 'Segmentation'], axis=1)
y_train=train['Segmentation']

In [27]:
# Hyperparameter tuning for KNN

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

leaf_size = list(range(1,30,3))
n_neighbors = list(range(1,20,2))
p=[1,2]

hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

In [28]:
knn = KNeighborsClassifier()

clf = GridSearchCV(knn, hyperparameters, cv=5)

In [29]:
best_model = clf.fit(X_train, y_train)

In [30]:
#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

Best leaf_size: 1
Best p: 1
Best n_neighbors: 17


In [31]:
X_test=true_test.drop('ID', axis=1)

pred_test=best_model.predict(X_test)
pred_test

array([1, 0, 0, 2, 1, 2, 2, 1, 2, 1, 1, 2, 3, 1, 3, 0, 1, 0, 0, 0, 2, 0,
       0, 3, 3, 1, 3, 0, 0, 2, 3, 1, 2, 3, 1, 1, 1, 2, 1, 3, 0, 0, 3, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 3, 1, 3, 3, 0, 0, 3, 3, 2, 3, 3, 0,
       3, 3, 1, 3, 2, 2, 0, 3, 0, 0, 3, 3, 0, 3, 0, 3, 3, 0, 0, 3, 3, 3,
       2, 0, 3, 1, 2, 3, 2, 1, 0, 2, 2, 1, 3, 0, 1, 3, 3, 0, 0, 3, 3, 0,
       0, 0, 2, 0, 3, 0, 0, 0, 2, 2, 3, 3, 3, 2, 2, 2, 0, 3, 1, 3, 2, 3,
       3, 2, 3, 0, 0, 1, 2, 0, 1, 3, 0, 3, 0, 2, 3, 3, 2, 2, 3, 3, 3, 3,
       2, 0, 3, 0, 3, 3, 1, 0, 3, 2, 0, 1, 3, 3, 1, 1, 0, 2, 3, 2, 0, 2,
       3, 0, 0, 1, 3, 1, 0, 2, 0, 1, 2, 0, 0, 3, 0, 1, 0, 2, 0, 2, 1, 0,
       0, 1, 1, 2, 1, 3, 2, 1, 3, 3, 0, 3, 2, 3, 1, 3, 3, 0, 3, 1, 3, 2,
       2, 3, 3, 3, 1, 3, 0, 0, 3, 0, 3, 2, 2, 0, 0, 0, 0, 3, 3, 1, 3, 0,
       3, 2, 3, 3, 0, 2, 0, 3, 0, 0, 3, 3, 1, 0, 1, 1, 3, 3, 2, 3, 1, 3,
       0, 3, 3, 3, 0, 0, 1, 3, 3, 1, 3, 0, 1, 0, 2, 0, 1, 3, 2, 1, 0, 1,
       3, 3, 3, 0, 3, 3, 3, 0, 0], dtype=int64)

In [32]:
true_test['Segmentation']=pred_test

In [33]:
true_test.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
6,459005,1,0,61,0,5,5.0,0,3.0,0,1
19,459045,0,0,88,0,7,1.0,1,4.0,0,0
32,459090,1,1,31,1,6,1.0,0,2.0,0,0
38,459116,1,0,60,0,6,2.0,1,5.0,0,2
43,459121,0,0,51,0,6,3.0,1,6.0,0,1


In [34]:
test_in_train = test[~test.index.isin(true_test.index)]
test_in_train

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,0,0,36,0,0,0.0,0,1.0,0
1,458994,1,0,37,0,1,8.0,1,4.0,0
2,458996,0,0,69,1,2,0.0,0,1.0,0
3,459000,1,0,59,1,3,11.0,2,2.0,0
4,459001,0,1,19,1,4,1.0,0,4.0,0
...,...,...,...,...,...,...,...,...,...,...
2620,467949,1,1,21,1,1,1.0,0,4.0,1
2621,467950,0,1,35,0,8,1.0,0,2.0,0
2622,467954,1,1,29,1,1,9.0,0,4.0,0
2623,467958,0,1,35,0,5,1.0,0,1.0,0


In [35]:
test_in_train['Segmentation']=false_test['Segmentation']

In [36]:
test_in_train

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,458989,0,0,36,0,0,0.0,0,1.0,0,1.0
1,458994,1,0,37,0,1,8.0,1,4.0,0,2.0
2,458996,0,0,69,1,2,0.0,0,1.0,0,0.0
3,459000,1,0,59,1,3,11.0,2,2.0,0,2.0
4,459001,0,1,19,1,4,1.0,0,4.0,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
2620,467949,1,1,21,1,1,1.0,0,4.0,1,3.0
2621,467950,0,1,35,0,8,1.0,0,2.0,0,3.0
2622,467954,1,1,29,1,1,9.0,0,4.0,0,3.0
2623,467958,0,1,35,0,5,1.0,0,1.0,0,0.0


In [37]:
full_test=pd.concat([test_in_train, true_test])
full_test

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,458989,0,0,36,0,0,0.0,0,1.0,0,1.0
1,458994,1,0,37,0,1,8.0,1,4.0,0,2.0
2,458996,0,0,69,1,2,0.0,0,1.0,0,0.0
3,459000,1,0,59,1,3,11.0,2,2.0,0,2.0
4,459001,0,1,19,1,4,1.0,0,4.0,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
2607,467913,1,0,48,0,6,3.0,0,1.0,0,3.0
2612,467930,1,0,45,0,8,4.0,1,2.0,0,3.0
2616,467938,1,0,46,0,8,5.0,0,1.0,5,3.0
2624,467960,0,1,53,0,8,1.0,0,2.0,0,0.0


In [38]:
sub1=full_test[['ID', 'Segmentation']]
sub1.head()

,ID,Segmentation
0,458989,1.0
1,458994,2.0
2,458996,0.0
3,459000,2.0
4,459001,2.0


In [39]:
sub1.shape

(2627, 2)

In [40]:
sub1['Segmentation']=sub1['Segmentation'].map({0:'A', 1:'B', 2:'C', 3:'D'})

In [41]:
sub1['Segmentation'].value_counts()

D    745
A    684
C    607
B    591
Name: Segmentation, dtype: int64

In [42]:
sub1.to_csv('Submission_2.csv', index=False)